In [ ]:
#!nvidia-smi

In [ ]:
!pip install ultralytics motrackers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from motrackers import CentroidTracker
from motrackers.utils import draw_tracks
import numpy as np
from ultralytics import YOLO
import time
from google.colab.patches import cv2 #Google Collab tidak bisa pakai cv2 yang original
from google.colab.patches import cv2_imshow
#import cv2

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
main_path = "/content/gdrive/MyDrive/Chicken_Health and_Behavior_Detection"
destination_path = '/content'

In [ ]:
!cp "$main_path/models/object_detection/model_yolov11s.zip" "$destination_path"

In [ ]:
!unzip -q "$destination_path/model_yolov11s.zip" -d "$destination_path/model"

In [ ]:
model = YOLO(f'{destination_path}/model/yolov11s.pt')

In [ ]:
video_path = f'{main_path}/data/processed/video_test.mp4'
output_path = f'{destination_path}/output_video_test.avi'

In [ ]:
# Open video
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Video writer
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))

# Tracker
tracker = CentroidTracker(max_lost=5, tracker_output_format='mot_challenge')
frame_count = 0

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.time()

    # Run YOLOv11 inference
    results = model(frame)

    # Get predictions
    detections = []
    colors = []

    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()       # Bounding box coordinates
        confidences = result.boxes.conf.cpu().numpy() # Confidence scores
        class_ids = result.boxes.cls.cpu().numpy()    # Class IDs

        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = box.astype(int)
            w = x2 - x1
            h = y2 - y1
            detections.append([x1, y1, w, h])
            colors.append((0, 255, 0))

    # Update tracker
    tracks = tracker.update(np.array(detections), confidences, class_ids)

    # Draw tracks
    tracked_frame = draw_tracks(frame, tracks)

    # Show Frame Slices
    #if frame_count % 10 == 0:
    #    cv2_imshow(tracked_frame)
    #    if cv2.waitKey(1) & 0xFF == ord('q'):
    #        break

    # Write to output video
    out.write(tracked_frame)
    frame_count += 1
    print(f"Processed frame {frame_count} | Detection + Tracking Time: {time.time() - start_time:.3f}s")

In [ ]:
cap.release()
out.release()
cv2.destroyAllWindows()